In [10]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


import matplotlib.pyplot as plt

Hyperparameters

In [2]:
input_size = 3  # Number of input features
hidden_size = 50 #50  # Number of LSTM units
output_size = 1  # Number of output features
num_layers = 2  # Number of LSTM layers

num_epochs = 400
learning_rate = 0.001
sequence_length = 10
batch_size = 32
dropout= 0.3 #0.3
weight_decay = 1e-5 #1e-5 # Adjust this value

#dataset control
left_only=False
no_negative = False
max_length_on = True
max_length = 20

Dataset

In [3]:
def create_sequences(x, y, seq_length):
    x = torch.tensor(x, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)
    x = torch.mul(x,1)
    y = torch.mul(y,1)
    sequences = []
    labels = []

    for i in range(len(x) - seq_length):
        if left_only and x[i+seq_length][2] != -1: # if direction != left
            continue
        if no_negative and x[i+seq_length][0] < 0: # if past goal
            continue
        if max_length_on and x[i+seq_length][-1] >= max_length: # 30 frames from crossing goal
            continue

        seq = x[i:i+seq_length]
        seq = seq[:,[0,1,3]] # Puck_cen_X, Puck_cen_Y
        label = y[i+seq_length]
        sequences.append(seq)
        labels.append(label)

    return torch.stack(sequences), torch.stack(labels)


Data Loader

In [4]:
class CustomDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

In [5]:
#df = pd.read_csv("Data/cleaned_data.csv")
#df = pd.read_csv("Data/Combined_numerated.csv")
df = pd.read_csv("Data/Final.csv")

X = df[["Puck_cen_X","Puck_cen_Y","direction","dt","Count"]].values # values between 2 and -1
Y = df[["Cross_Left"]].values
#Y = df["Cross_right"].values

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, shuffle=False)
X_test, X_train, y_test, y_train = train_test_split(X, Y, test_size=0.7, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.96, shuffle=False)

train_sequences, train_labels = create_sequences(X_train, y_train, sequence_length)
val_sequences, val_labels = create_sequences(X_val, y_val, sequence_length)
test_sequences, test_labels = create_sequences(X_test, y_test, sequence_length)

In [6]:
train_dataset = CustomDataset(train_sequences, train_labels)
val_dataset = CustomDataset(val_sequences, val_labels)
test_dataset = CustomDataset(test_sequences, test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
print("train:",train_sequences.shape)
print("val",val_sequences.shape)
print("test",test_sequences.shape)

train: torch.Size([8393, 10, 3])
val torch.Size([108, 10, 3])
test torch.Size([2664, 10, 3])


In [8]:
del X, Y, X_train, X_test, X_val, y_train, y_test, y_val # deleate
del train_sequences, val_sequences

Physics Model

In [33]:
from LSTM_Prediction import LSTM_Prediction
from Physics_Prediction import physics_prediction
LSTM = LSTM_Prediction(path= "LSTM_HS80_L2_dt.pt",dt = True)
Phys = Physics_Prediction(dt = True)

AttributeError: '_IncompatibleKeys' object has no attribute 'to'

In [ ]:
sequence = test_sequences[0]
phys_pred = 
